In [ ]:
!pip install -U dspy
!pip install -U datasets # HuggingFace Datasets for Fine-tuning
from google.colab import userdata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.7/261.7 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.3/272.3 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 17.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Succe

In [ ]:
import dspy
OPENAI_API_KEY=userdata.get("OPENAI_API_KEY")
HF_TOKEN=userdata.get("HF_TOKEN")

lm = dspy.LM("openai/gpt-4o-mini",
             api_key=OPENAI_API_KEY,)
dspy.configure(lm=lm)

## Math

In [ ]:
math = dspy.ChainOfThought("question -> answer: float")
math(question="Two dice are tossed. What is the probability that the sum equals two?")

Prediction(
    reasoning='When two dice are tossed, each die has 6 faces, resulting in a total of 6 * 6 = 36 possible outcomes. The only way to achieve a sum of 2 is if both dice show a 1 (1, 1). There is only 1 favorable outcome for this event. Therefore, the probability of the sum equaling 2 is the number of favorable outcomes divided by the total number of outcomes, which is 1/36.',
    answer=0.027777777777777776
)

## RAG

In [ ]:
def search_wikipedia(query: str) -> list[str]:
    try:
        results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(query, k=3)
        # Check if the results structure is as expected before accessing 'text'
        if results and isinstance(results, list) and all('text' in x for x in results):
            return [x["text"] for x in results]
        else:
            print("Warning: 'text' key not found in all results or results format unexpected.")
            return [] # Return empty list or handle as appropriate
    except KeyError as e:
        print(f"Error accessing expected key: {e}")
        print("The response from the ColBERTv2 server did not contain the expected 'topk' key.")
        return [] # Return an empty list or handle the error as appropriate
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []


rag = dspy.ChainOfThought("context, question -> response")

# question = "What's the name of the castle that David Gregory inherited?"
question = "How long did Steve Jobs run Apple for?"
rag(context=search_wikipedia(question), question=question)

Error accessing expected key: 'topk'
The response from the ColBERTv2 server did not contain the expected 'topk' key.


Prediction(
    reasoning='Steve Jobs co-founded Apple Inc. in 1976 and played a significant role in the company until he was ousted in 1985. He returned to Apple in 1997 and remained with the company until his death in 2011. Therefore, he ran Apple for approximately 9 years during his second tenure and about 9 years during his first tenure, totaling around 18 years.',
    response='Steve Jobs ran Apple for a total of approximately 18 years, with his first tenure from 1976 to 1985 and his second from 1997 until his death in 2011.'
)

## Classification

In [ ]:
from typing import Literal

class Classify(dspy.Signature):
    """Classify sentiment of a given sentence."""

    sentence: str = dspy.InputField()
    sentiment: Literal["positive", "negative", "neutral"] = dspy.OutputField()
    confidence: float = dspy.OutputField()

classify = dspy.Predict(Classify)
classify(sentence="This book was super fun to read, though not the last chapter.")

Prediction(
    sentiment='positive',
    confidence=0.85
)

## Information Extraction

In [ ]:
class ExtractInfo(dspy.Signature):
    """Extract structured information from text."""

    text: str = dspy.InputField()
    title: str = dspy.OutputField()
    headings: list[str] = dspy.OutputField()
    entities: list[dict[str, str]] = dspy.OutputField(desc="a list of entities and their metadata")

module = dspy.Predict(ExtractInfo)

text = "Apple Inc. announced its latest iPhone 14 today." \
    "The CEO, Tim Cook, highlighted its new features in a press release."
response = module(text=text)

print(response.title)
print(response.headings)
print(response.entities)

Apple Inc. Announces iPhone 14
['Announcement', 'CEO Statement', 'New Features']
[{'name': 'Apple Inc.', 'type': 'Organization'}, {'name': 'iPhone 14', 'type': 'Product'}, {'name': 'Tim Cook', 'type': 'Person'}]


## Agents

In [ ]:
def evaluate_math(expression: str):
    return dspy.PythonInterpreter({}).execute(expression)

def search_wikipedia(query: str):
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(query, k=3)
    return [x["text"] for x in results]

react = dspy.ReAct("question -> answer: float", tools=[evaluate_math, search_wikipedia])

pred = react(question="What is 9362158 divided by the year of birth of David Gregory of Kinnairdy castle?")
print(pred.answer)

5765.0


## Mult-Stage Pipeline

In [ ]:

class Outline(dspy.Signature):
    """Outline a thorough overview of a topic."""

    topic: str = dspy.InputField()
    title: str = dspy.OutputField()
    sections: list[str] = dspy.OutputField()
    section_subheadings: dict[str, list[str]] = dspy.OutputField(desc="mapping from section headings to subheadings")

class DraftSection(dspy.Signature):
    """Draft a top-level section of an article."""

    topic: str = dspy.InputField()
    section_heading: str = dspy.InputField()
    section_subheadings: list[str] = dspy.InputField()
    content: str = dspy.OutputField(desc="markdown-formatted section")

class DraftArticle(dspy.Module):
    def __init__(self):
        self.build_outline = dspy.ChainOfThought(Outline)
        self.draft_section = dspy.ChainOfThought(DraftSection)

    def forward(self, topic):
        outline = self.build_outline(topic=topic)
        sections = []
        for heading, subheadings in outline.section_subheadings.items():
            section, subheadings = f"## {heading}", [f"### {subheading}" for subheading in subheadings]
            section = self.draft_section(topic=outline.title, section_heading=section, section_subheadings=subheadings)
            sections.append(section.content)
        return dspy.Prediction(title=outline.title, sections=sections)

draft_article = DraftArticle()
article = draft_article(topic="Apocolyptic 2012 Theory")

In [ ]:
print(article)

Prediction(
    title='The Apocalyptic 2012 Theory: Origins, Impact, and Aftermath',
    sections=['## Origins of the 2012 Theory\n\n### Mayan Calendar and Cosmology\nThe Mayan civilization, known for its advanced understanding of astronomy and mathematics, developed a complex calendar system that included several cycles. One of the most significant cycles is the Long Count calendar, which spans approximately 5,125.36 years. This calendar is often cited as having a "completion" date of December 21, 2012. Many interpreted this date as a prophecy of an impending apocalypse, despite the fact that the Mayans themselves did not predict an end to the world but rather a transition to a new era.\n\n### Interpretations of End Dates\nThe interpretation of the Mayan calendar\'s end date has varied widely. Some scholars and enthusiasts suggested that it signified a cataclysmic event, while others argued it represented a time of renewal and transformation. This divergence in interpretation fueled s

## Optimising Prompts for a ReAct Agent

In [ ]:
import dspy
from dspy.datasets import HotPotQA

# dspy.configure(lm=dspy.LM("openai/gpt-4o-mini", api_key=OPENAI_API_KEY))

def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(query, k=3)
    # Check if the results structure is as expected before accessing 'text'
    if results and isinstance(results, list) and all('text' in x for x in results):
        return [x["text"] for x in results]
    else:
        print("Warning: 'text' key not found in all results or results format unexpected.")
        return [] # Return empty list or handle as appropriate


trainset = [x.with_inputs('question') for x in HotPotQA(train_seed=2024, train_size=100).train]
react = dspy.ReAct("question -> answer", tools=[search_wikipedia])

# tp = dspy.MIPROv2(metric=dspy.evaluate.answer_exact_match, auto="light", num_threads=24) # if using Local LLM 24 is possible
tp = dspy.MIPROv2(metric=dspy.evaluate.answer_exact_match, auto="light", num_threads=4) # 4 is ideal for API Calls
optimized_react = tp.compile(react, trainset=trainset)

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'hotpot_qa' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'hotpot_qa' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


README.md: 0.00B [00:00, ?B/s]

fullwiki/train-00000-of-00002.parquet:   0%|          | 0.00/166M [00:00<?, ?B/s]

fullwiki/train-00001-of-00002.parquet:   0%|          | 0.00/166M [00:00<?, ?B/s]

fullwiki/validation-00000-of-00001.parqu(…):   0%|          | 0.00/28.0M [00:00<?, ?B/s]

...

Average Metric: 14.00 / 25 (56.0%):  71%|███████▏  | 25/35 [02:49<01:17,  7.79s/it]

2025/10/03 17:12:08 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: 17.00 / 31 (54.8%):  91%|█████████▏| 32/35 [03:11<00:11,  3.67s/it]

2025/10/03 17:12:37 ERROR dspy.utils.parallelizer: Error for Example({'question': "How many times has the driver, who won the Nation's Cup with Petter Solberg, in the 2014 Race of Champions, won  the 24 Hours of Le Mans ?", 'answer': 'nine'}) (input_keys={'question'}): litellm.RateLimitError: RateLimitError: OpenAIException - Rate limit reached for gpt-4o-mini in organization org-8PmqQww7DKKRrI6VcljnZDsz on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.. Set `provide_traceback=True` for traceback.


Average Metric: 17.00 / 33 (51.5%): 100%|██████████| 35/35 [03:59<00:00,  8.91s/it]

An informal run like this raises ReAct's score from 24% to 51%, by teaching gpt-4o-mini more about the specifics of the task.

## Optimising Prompts for a RAG Agent

In [ ]:
# Optimising Prompts for a RAG Agent

class RAG(dspy.Module):
    def __init__(self, num_docs=5):
        self.num_docs = num_docs
        self.respond = dspy.ChainOfThought("context, question -> response")

    def forward(self, question):
        context = search(question, k=self.num_docs)   # defined in tutorial linked below
        return self.respond(context=context, question=question)

# tp = dspy.MIPROv2(metric=dspy.evaluate.SemanticF1(decompositional=True), auto="medium", num_threads=24)
tp = dspy.MIPROv2(metric=dspy.evaluate.SemanticF1(decompositional=True), auto="medium", num_threads=4)
optimized_rag = tp.compile(RAG(), trainset=trainset, max_bootstrapped_demos=2, max_labeled_demos=2)

## Optimising Weights for Classification

In [ ]:
import random
from typing import Literal

from datasets import load_dataset

import dspy
from dspy.datasets import DataLoader

# Load the Banking77 dataset.
CLASSES = load_dataset("PolyAI/banking77", split="train", trust_remote_code=True).features["label"].names
kwargs = {"fields": ("text", "label"), "input_keys": ("text",), "split": "train", "trust_remote_code": True}

# Load the first 2000 examples from the dataset, and assign a hint to each *training* example.
trainset = [
    dspy.Example(x, hint=CLASSES[x.label], label=CLASSES[x.label]).with_inputs("text", "hint")
    for x in DataLoader().from_huggingface(dataset_name="PolyAI/banking77", **kwargs)[:2000]
]
random.Random(0).shuffle(trainset)

In [ ]:
import dspy
# lm=dspy.LM('openai/gpt-4o-mini-2024-07-18')

# Define the DSPy module for classification. It will use the hint at training time, if available.
signature = dspy.Signature("text, hint -> label").with_updated_fields("label", type_=Literal[tuple(CLASSES)])
classify = dspy.ChainOfThought(signature)
classify.set_lm(lm)

# Optimize via BootstrapFinetune.
optimizer = dspy.BootstrapFinetune(metric=(lambda x, y, trace=None: x.label == y.label), num_threads=24)
optimized = optimizer.compile(classify, trainset=trainset)

optimized(text="What does a pending cash withdrawal mean?")

# For a complete fine-tuning tutorial, see: https://dspy.ai/tutorials/classification_finetuning/

An informal run similar to this on DSPy 2.5.29 raises GPT-4o-mini's score 66% to 87%.